In [2]:
%matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
filename = "/home/qiwsir/Documents/DataAnalysis/chapter05/snell.csv"     #数据地址
datas = pd.read_csv(filename)
datas.head()

Using matplotlib backend: TkAgg


,alpha,belta
0,0.001745,0.001164
1,0.017577,0.011717
2,0.033408,0.022270
3,0.049239,0.032819
4,0.065071,0.043364


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    datas['alpha'], datas['belta'], test_size=0.4, random_state=40)

In [4]:
datas['alpha'].count(), X_train.count(), X_test.count()

(100, 60, 40)

In [5]:
plt.scatter(X_train, Y_train)

In [6]:
import statsmodels.api as sm    #①
X = pd.DataFrame({"c": np.ones(60).T, "x": X_train})    #②
model = sm.OLS(Y_train, X)     #③
result = model.fit()    #④
result.params     #⑤

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


c    0.075643
x    0.471903
dtype: float64

In [7]:
coe = result.params
fig, ax = plt.subplots()
ax.scatter(X_train, Y_train)
x = np.linspace(0, 1.8, 100)
y = coe['c'] + coe['x']*x
ax.plot(x, y, color='red')

In [8]:
 XX_train = pd.DataFrame({"c": np.ones(60).T, "x":X_train, "x**2": X_train*X_train})
result2 = sm.OLS(Y_train, XX_train).fit()
result2.params

c      -0.022771
x       0.820724
x**2   -0.210711
dtype: float64

In [9]:
coe = result2.params
fig, ax = plt.subplots()
ax.scatter(X_train, Y_train)
x = np.linspace(0, 1.8, 100)
y = coe['c'] + coe['x']*x + coe['x**2']*x*x
ax.plot(x, y, color='red')


In [10]:
coe = result.params
y = coe['c'] + coe['x']*X_test
np.corrcoef(y, Y_test)    #计算相关系数


array([[ 1.        ,  0.99252036],
       [ 0.99252036,  1.        ]])

In [11]:
coe = result2.params
y = coe['c'] + coe['x']*X_test + coe['x**2']*X_test*X_test
np.corrcoef(y, Y_test)


array([[ 1.        ,  0.99929732],
       [ 0.99929732,  1.        ]])

In [12]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  belta   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.965
Method:                 Least Squares   F-statistic:                     1647.
Date:                Thu, 01 Feb 2018   Prob (F-statistic):           2.80e-44
Time:                        15:21:17   Log-Likelihood:                 105.99
No. Observations:                  60   AIC:                            -208.0
Df Residuals:                      58   BIC:                            -203.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c              0.0756      0.011      6.855      0.000       0.054       0.098
x              0.4719      0.012     40.578      0.000       0.449       0.495
==============================================================================
Omnibus:                       12.947   Durbin-Watson:                   1.327
Prob(Omnibus):                  0.002   Jarque-Bera (JB):                4.770
Skew:                          -0.407   Prob(JB):                       0.0921
Kurtosis:                       1.884   Cond. No.                         3.81
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
print(result.rsquared, result2.rsquared)

0.965974637844 0.998641952133


In [14]:
import scipy as sp
z, p = sp.stats.normaltest(result2.resid.values)
p

0.12456375865335403

In [15]:
import statsmodels.graphics.api as smg
fig, ax = plt.subplots()
smg.qqplot(result2.resid, ax=ax)
fig.tight_layout()


In [17]:
import seaborn as sns
sns.residplot(x=X_train.values, y=result2.resid.values)


In [18]:
err = np.random.RandomState(1)
x = 10 * err.rand(200)
y = 2 * x - 5 + err.randn(200)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=40)


In [20]:
from sklearn.linear_model import LinearRegression    #①
model = LinearRegression(fit_intercept=True)    #②
model.fit(x_train[:, np.newaxis], y_train)    #③

print("slope: ", model.coef_[0])
print("intercept: ", model.intercept_)


slope:  1.99154872449
intercept:  -4.88227590023


In [21]:
y_fit = model.predict(x_test[:, np.newaxis])     #①
np.corrcoef(y_fit, y_test)


array([[ 1.        ,  0.98499746],
       [ 0.98499746,  1.        ]])

In [22]:
resid = y_test - y_fit
sns.residplot(x=x_test, y=resid)


In [23]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(3, include_bias=True)


In [24]:
x = np.array([2, 3, 4])
poly.fit_transform(x[:, None])


array([[  1.,   2.,   4.,   8.],
       [  1.,   3.,   9.,  27.],
       [  1.,   4.,  16.,  64.]])

In [25]:
poly = PolynomialFeatures(2, interaction_only=True)
x = pd.DataFrame({"x1": [1, 2, 3], "x2": [4, 5, 6]})
poly.fit_transform(x)


array([[  1.,   1.,   4.,   4.],
       [  1.,   2.,   5.,  10.],
       [  1.,   3.,   6.,  18.]])

In [26]:
err = np.random.RandomState(1)
x = 10 * err.rand(100)
y = np.sin(x) + 0.1 * err.randn(100)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=40)


In [27]:
from sklearn.pipeline import Pipeline    #①
model = Pipeline([('poly', PolynomialFeatures(degree=7)), 
                  ('linear', LinearRegression(fit_intercept=False))])     #②

model = model.fit(x_train[:, np.newaxis], y_train)
y_fit = model.predict(x_test[:, np.newaxis])

np.corrcoef(y_fit, y_test)


array([[ 1.        ,  0.98994052],
       [ 0.98994052,  1.        ]])

In [28]:
import math
math.sin(math.pi/2)


1.0

In [29]:
model.predict([[math.pi/2]])

array([ 1.03096889])

In [30]:
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

class FooTransformer(TransformerMixin):
    def __init__(self, number):
        self.number = number

    def fit(self, X, y=None):
        print("Transformer No.{}: fit X={}".format(self.number, X))
        return self

    def transform(self, X):
        print("Transformer No.{}: transform X={} => X+10={}".format(self.number, X, X+10))
        return X+10

class BarEstimator(BaseEstimator):
    def __init__(self, number):
        self.number = number

    def fit(self, X, y=None):
        print("Estimator No.{}: fit X={}".format(self.number, X))
        return self

    def predict(self, X):
        print("Estimator No.{}: predict X={} is...".format(self.number, X))
        return "a new result"
p1 = Pipeline(steps=[("Trans1", FooTransformer(1)),
                     ("Trans2", FooTransformer(2)),
                     ("Estmt1", BarEstimator(1))])
print(p1)

p2 = make_pipeline(FooTransformer(1), FooTransformer(2), BarEstimator(1))
print(p2)


Pipeline(memory=None,
     steps=[('Trans1', <__main__.FooTransformer object at 0x7f16448e5b00>), ('Trans2', <__main__.FooTransformer object at 0x7f16448e5ac8>), ('Estmt1', BarEstimator(number=1))])
Pipeline(memory=None,
     steps=[('footransformer-1', <__main__.FooTransformer object at 0x7f16448e5860>), ('footransformer-2', <__main__.FooTransformer object at 0x7f16448e5588>), ('barestimator', BarEstimator(number=1))])


In [31]:
p = Pipeline(steps=[("Trans1", FooTransformer(1)),
                     ("Trans2", FooTransformer(2)),
                     ("Estmt1", BarEstimator(1))])
print("#===== Pipeline fitting =====#")
p.fit(X=100)
print("#===== Pipeline predicting =====#")
pred = p.predict(X=100)
print(pred)


#===== Pipeline fitting =====#
Transformer No.1: fit X=100
Transformer No.1: transform X=100 => X+10=110
Transformer No.2: fit X=110
Transformer No.2: transform X=110 => X+10=120
Estimator No.1: fit X=120
#===== Pipeline predicting =====#
Transformer No.1: transform X=100 => X+10=110
Transformer No.2: transform X=110 => X+10=120
Estimator No.1: predict X=120 is...
a new result
